<a href="https://colab.research.google.com/github/sohaildua/PysparkLearning/blob/main/Manipulate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive --quiet 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 49.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=426275a161b60211a78d6626566226a676f0f46fc15f15c1c976e308c588d665
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [419]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Manipulate").getOrCreate()
spark

In [420]:
from pyspark.sql.functions import concat_ws,col
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
df2=df.select(concat_ws('_!_',df.firstname,df.middlename,df.lastname)
              .alias("FullName"),"dob","gender","salary")
df2.show(truncate=False)

+--------------------+----------+------+------+
|FullName            |dob       |gender|salary|
+--------------------+----------+------+------+
|James_!__!_Smith    |1991-04-01|M     |3000  |
|Michael_!_Rose_!_   |2000-05-19|M     |4000  |
|Robert_!__!_Williams|1978-09-05|M     |4000  |
|Maria_!_Anne_!_Jones|1967-12-01|F     |4000  |
|Jen_!_Mary_!_Brown  |1980-02-17|F     |-1    |
+--------------------+----------+------+------+



In [421]:
videos = spark.read.csv('youtubevideos.csv', inferSchema=True, header=True)

In [422]:
videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [423]:
videos.show(5, False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [424]:
videos.limit(4).toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...


In [425]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [426]:

df_videos = videos.withColumn("views",videos["views"].cast(IntegerType())).\
withColumn("dislikes",videos["dislikes"].cast(IntegerType())).\
withColumn("likes",videos.likes.cast(IntegerType())).\
withColumn("trending_date",to_date(videos.trending_date,'yy.dd.mm'))
#withColumn("publish_time",to_timestamp(videos.publish_time,'yyyy-MM-dd HH:mm:ss')).\

In [427]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [428]:
df_videos.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|   2017-01-14|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [429]:
df_videos = df_videos.withColumn('publish_time_2',regexp_replace(df_videos.publish_time, 'T', ' '))

In [430]:
df_videos.show(4,False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [431]:

df_videos = df_videos.withColumn('publish_time_2',regexp_replace(df_videos.publish_time_2, 'Z', ' '))
df_videos=df_videos.withColumnRenamed("publish_time","publish_time_extra")

In [432]:
df_videos.show(4,False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [433]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

df_videos = df_videos.withColumn("publish_time_3",to_timestamp(df_videos.publish_time_2,'yyyy-MM-dd HH:mm:ss.SSS'))

In [434]:
df_videos.show(1)

+-----------+-------------+--------------------+-------------+-----------+--------------------+---------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------------+
|   video_id|trending_date|               title|channel_title|category_id|  publish_time_extra|           tags| views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|      publish_time_2|     publish_time_3|
+-----------+-------------+--------------------+-------------+-----------+--------------------+---------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------------+
|2kyS6SvSYSE|   2017-01-14|WE WANT TO TALK A...| CaseyNeistat|         22|2017-11-13T17:13:...|SHANtell martin|748374|57527|    2966|

In [435]:
df_videos=df_videos.withColumnRenamed("publish_time_3","publish_time")

In [436]:
df_videos.show(4,False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [437]:
df_videos.select("publish_time").show(4,False)

+-------------------+
|publish_time       |
+-------------------+
|2017-11-13 17:13:01|
|2017-11-13 07:30:00|
|2017-11-12 19:05:24|
|2017-11-13 11:00:04|
+-------------------+
only showing top 4 rows



In [438]:
df = spark.createDataFrame([["Alex", 20], ["Bob", 30], ["Cathy", 40]], ["name", "age"])
df.show()

+-----+---+
| name|age|
+-----+---+
| Alex| 20|
|  Bob| 30|
|Cathy| 40|
+-----+---+



In [439]:

df.select(translate("name", "Aeo", "#@%")).show()

+-------------------------+
|translate(name, Aeo, #@%)|
+-------------------------+
|                     #l@x|
|                      B%b|
|                    Cathy|
+-------------------------+



In [440]:
df_videos.select("title").show(4,False)

+--------------------------------------------------------------+
|title                                                         |
+--------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                            |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)|
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons         |
|Nickelback Lyrics: Real or Fake?                              |
+--------------------------------------------------------------+
only showing top 4 rows



In [441]:
df_videos = df_videos.withColumn("title",trim(df_videos.title))

In [411]:
df_videos.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------------+
|   video_id|trending_date|               title|       channel_title|category_id|  publish_time_extra|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|      publish_time_2|       publish_time|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------------+
|2kyS6SvSYSE|   2017-01-14|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13

In [442]:
df_videos.withColumn("title",lower(df_videos.title)).show(4,False)


+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [443]:
df=df_videos

In [446]:
#when-otherwise operator 
#new one for me
#end case
df_videos.select("likes","dislikes",(when(df_videos.likes>df_videos.dislikes,"good").when(df_videos.likes<df_videos.dislikes,"bad").otherwise("adad")).alias("asa")).show(3)

+------+--------+----+
| likes|dislikes| asa|
+------+--------+----+
| 57527|    2966|good|
| 97185|    6146|good|
|146033|    5339|good|
+------+--------+----+
only showing top 3 rows



In [448]:
df.selectExpr("likes","dislikes","CASE WHEN likes > dislikes THEN 'GOOD' WHEN likes < dislikes THEN 'BAD' ELSE 'NOGH_JOS' END AS ajsaskl").show(3)

+------+--------+-------+
| likes|dislikes|ajsaskl|
+------+--------+-------+
| 57527|    2966|   GOOD|
| 97185|    6146|   GOOD|
|146033|    5339|   GOOD|
+------+--------+-------+
only showing top 3 rows



In [449]:
#concat_ws to conticate data

In [451]:
df.select("trending_date", year("trending_date"), month("trending_date")).show()

+-------------+-------------------+--------------------+
|trending_date|year(trending_date)|month(trending_date)|
+-------------+-------------------+--------------------+
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2

In [454]:
df.select("trending_date", year("trending_date"), month("trending_date"),"publish_time",datediff(df.publish_time,df.trending_date)).show()

+-------------+-------------------+--------------------+-------------------+-------------------------------------+
|trending_date|year(trending_date)|month(trending_date)|       publish_time|datediff(publish_time, trending_date)|
+-------------+-------------------+--------------------+-------------------+-------------------------------------+
|   2017-01-14|               2017|                   1|2017-11-13 17:13:01|                                  303|
|   2017-01-14|               2017|                   1|2017-11-13 07:30:00|                                  303|
|   2017-01-14|               2017|                   1|2017-11-12 19:05:24|                                  302|
|   2017-01-14|               2017|                   1|2017-11-13 11:00:04|                                  303|
|   2017-01-14|               2017|                   1|2017-11-12 18:01:41|                                  302|
|   2017-01-14|               2017|                   1|2017-11-13 19:07:23|    

In [459]:
array = df.select("title", split(df.title,' ').alias("data"))

In [460]:
array.show(121,False)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|title                                                                                               |data                                                                                                                  |
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                                                                  |[WE, WANT, TO, TALK, ABOUT, OUR, MARRIAGE]                                                                            |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)                                      |[The, Trum

In [462]:
array.select('title',array_contains(array.data,"ABOUT")).show()

+--------------------+---------------------------+
|               title|array_contains(data, ABOUT)|
+--------------------+---------------------------+
|WE WANT TO TALK A...|                       true|
|The Trump Preside...|                      false|
|Racist Superman |...|                      false|
|Nickelback Lyrics...|                      false|
|I Dare You: GOING...|                      false|
|2 Weeks with iPho...|                      false|
|Roy Moore & Jeff ...|                      false|
|5 Ice Cream Gadge...|                      false|
|The Greatest Show...|                      false|
|Why the rise of t...|                      false|
|Dion Lewis' 103-Y...|                      false|
|(SPOILERS) 'Shiva...|                      false|
|Marshmello - Bloc...|                      false|
|Which Countries A...|                      false|
|SHOPPING FOR NEW ...|                      false|
|    The New SpotMini|                      false|
|One Change That W...|         

In [463]:
array.select('title',array_distinct(array.data)).show()

+--------------------+--------------------+
|               title|array_distinct(data)|
+--------------------+--------------------+
|WE WANT TO TALK A...|[WE, WANT, TO, TA...|
|The Trump Preside...|[The, Trump, Pres...|
|Racist Superman |...|[Racist, Superman...|
|Nickelback Lyrics...|[Nickelback, Lyri...|
|I Dare You: GOING...|[I, Dare, You:, G...|
|2 Weeks with iPho...|[2, Weeks, with, ...|
|Roy Moore & Jeff ...|[Roy, Moore, &, J...|
|5 Ice Cream Gadge...|[5, Ice, Cream, G...|
|The Greatest Show...|[The, Greatest, S...|
|Why the rise of t...|[Why, the, rise, ...|
|Dion Lewis' 103-Y...|[Dion, Lewis', 10...|
|(SPOILERS) 'Shiva...|[(SPOILERS), 'Shi...|
|Marshmello - Bloc...|[Marshmello, -, B...|
|Which Countries A...|[Which, Countries...|
|SHOPPING FOR NEW ...|[SHOPPING, FOR, N...|
|    The New SpotMini|[The, New, SpotMini]|
|One Change That W...|[One, Change, Tha...|
|How does your bod...|[How, does, your,...|
|HomeMade Electric...|[HomeMade, Electr...|
|Founding An Inbre...|[Founding,